In [1]:
import pandas as pd
## https://gist.github.com/michhar/2dfd2de0d4f8727f873422c5d959fff5

data = pd.read_csv('./data/Titanic.csv', sep='\t')
data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
not_survived = data[data['Survived'] == 0].shape[0]
survived = data[data['Survived'] == 1].shape[0]

print('# negative labels: ', not_survived)
print('# positive labels: ', survived)
print('base prevalence of label: ', survived/(data.shape[0]))

# negative labels:  102
# positive labels:  54
base prevalence of label:  0.34615384615384615


In [3]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.33)

train_X = train.iloc[:,1:]
train_y = train['Survived']

test_X = test.iloc[:,1:]
test_y = test['Survived']

print('Train data shape: ', train_X.shape)
print('Train labels shape: ', train_y.shape)
print('Test data shape: ', test_X.shape)
print('Test labels shape: ', test_y.shape)

Train data shape:  (104, 11)
Train labels shape:  (104,)
Test data shape:  (52, 11)
Test labels shape:  (52,)


In [2]:
# import matplotlib.pyplot as plt
# %matplotlib inline

# # gca stands for 'get current axis'
# ax = plt.gca()

# df.plot(kind='line',x='Age',y='Survived',ax=ax)
# df.plot(kind='line',x='Fare',y='Survived', color='red', ax=ax)

# plt.show()

1. simple imputer (you decide the strategy)
2. label encoder
3. one hot encoder
4. standard scaler

### Column Transformer

Use ColumnTransformer any time you have features in your dataframe that need pre-processing.

In [8]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_transformer

pipeline = Pipeline(steps=[('si', SimpleImputer(strategy='mean'))])
    
print(pipeline.steps)

[('si', SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='mean', verbose=0))]


In [49]:



## for ['Age', 'Fare']
p1 = Pipeline([
    ('impute', SimpleImputer(strategy='mean')),
    ('scale', StandardScaler())
])

## for ['Siblings/Spouses Aboard', 'P/C Aboard', 'Pclass']
p2 = Pipeline([
    ('impute_2', SimpleImputer(strategy='median'))
])

# sex, pclass 
p3 = Pipeline([
    ('encode', OneHotEncoder())
])

ct1 = ColumnTansformer([
    ('num', p1, ['Age', 'Fare']),
    ('num2', p2, ['Siblings/Spouses Aboard', 'P/C Aboard', 'Pclass'])
])

ct2 = ColumnTransformer([
    ('p3', p3, ['Sex', 'Pclass'])
])

pipe = FeatureUnion([
    transformer_list=[('c1', ct1), ('c2', ct2)]
])


,0,1,2,3,4,5,6
0,58,29.7,0,0,1,Mr. Edward Austin Kent,male
1,22,8.05,0,0,3,Mr. Charles Joseph Shorney,male
2,22,7.5208,0,0,3,Mr. Nils August Karlsson,male
3,18,14.4583,1,0,3,Mrs. Joseph (Maria Elias) Caram,female
4,60,26.55,0,0,1,Col. John Weir,male
...,...,...,...,...,...,...,...
589,34.5,6.4375,0,0,3,Mr. Peter L Lemberopolous,male
590,20,7.925,0,0,3,Mr. Ilmari Rudolf Alhomaki,male
591,18,15.5,1,0,3,Miss. Katherine Murphy,female
592,34,13,0,0,2,Mr. Martin Ponesell,male


In [ ]:
preprocess = make_column_transformer(
    (SimpleImputer(strategy='mean'), ['Age', 'Fare']),
    (SimpleImputer(strategy='median'), ['Siblings/Spouses Aboard', 'Parents/Children Aboard', 'Pclass']),
    remainder='passthrough'
)

imputation_pipe = Pipeline(steps=[
    ('ct_impute', preprocess)
])

ct_2 = make_column_transformer(
    (StandardScaler(), ['Age', 'Fare']),
    (OneHotEncoder(), ['Pclass', 'Sex']), remainder='passthrough'
)

transforms = Pipeline(steps=[
    ('num_cat', ct_2)
])

pipe = FeatureUnion(transformer_list=[('imp', imputation_pipe), ('t', transforms)])


pd.DataFrame(preprocess.fit_transform(train_X))

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

imp = SimpleImputer(strategy='mean')
ohe = OneHotEncoder()
sc = StandardScaler

pipe = Pipeline([
    ('ohe', ohe, ['Sex', 'Pclass']),
    ('imp', imp),
    ('sc', sc)
        ])

pipe.fit(train)

# pipe.transform(train).toarray()